In [1]:
# Importing stock ml libraries
import warnings
warnings.simplefilter('ignore')
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import DistilBertTokenizer, DistilBertModel
import logging
logging.basicConfig(level=logging.ERROR)

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchmetrics import F1Score

In [ ]:
# # Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [4]:
data = pd.read_csv('/kaggle/input/dataset1/Distill_Try.csv')

In [5]:
d=[]
for x in data['Tag']:
    x=x[1:-1]
    x=x.replace("'","")
    y=x.split(",")
    d.append(y)
for x in d:
    if(len(x)>1):
        for i in range(1,len(x)):
            x[i]=x[i][1:]

In [6]:
from sklearn.preprocessing import MultiLabelBinarizer
one_hot = MultiLabelBinarizer()
a=one_hot.fit_transform(d)
b = one_hot.classes_
xt = data['Text']
print(one_hot.classes_)

['cs.AI' 'cs.AR' 'cs.CE' 'cs.CL' 'cs.CR' 'cs.CV' 'cs.DB' 'cs.DC' 'cs.DM'
 'cs.GT' 'cs.IR' 'cs.IT' 'cs.LG' 'cs.LO' 'cs.NI' 'cs.OS' 'cs.PL' 'cs.RO'
 'cs.SD' 'cs.SE' 'econ.EM' 'econ.GN' 'econ.TH' 'eess.AS' 'eess.IV'
 'eess.SP' 'math.AC' 'math.AP' 'math.AT' 'math.CO' 'math.CV' 'math.GR'
 'math.IT' 'math.LO' 'math.NT' 'math.PR' 'math.QA' 'math.ST' 'q-bio.BM'
 'q-bio.CB' 'q-bio.GN' 'q-bio.MN' 'q-bio.NC' 'q-bio.TO' 'q-fin.CP'
 'q-fin.EC' 'q-fin.GN' 'q-fin.MF' 'q-fin.PM' 'q-fin.PR' 'q-fin.RM'
 'q-fin.TR' 'stat.AP' 'stat.CO' 'stat.ME' 'stat.ML' 'stat.TH']


In [7]:
data['labels']= pd.DataFrame({'labels':[np.array(row) for row in a]})

In [8]:
data.head()

,Id,Text,Tag,Tags,labels
0,9707,axiomatic aspects default inference \n axioms ...,['cs.LO'],[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
1,24198,"extensions group infinite conjugacy classes, i...",['math.GR'],[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,35766,analysis complex-valued cnns rf data-driven wi...,"['cs.LG', 'cs.IT', 'eess.SP', 'math.IT']",[0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, ..."
3,14322,reconstruction drift diffusion from transition...,"['math.PR', 'math.ST', 'stat.TH']",[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,709,three classes propagation rules grs egrs codes...,"['cs.IT', 'math.IT']",[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."


In [9]:
data.drop(['Id'], inplace=True, axis=1)
data.drop(['Tag'], inplace = True, axis = 1)

In [10]:
new_df = pd.DataFrame()
new_df['text'] = data['Text']
new_df['labels'] = data['labels']

In [11]:
for i in range(51210):
    new_df['labels'][i]=new_df['labels'][i].tolist()

In [12]:
type(new_df['labels'][1])

list

In [13]:
# Sections of config

# Defining some key variables that will be used later on in the training
MAX_LEN = 500
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4
EPOCHS = 8
LEARNING_RATE = 1e-05

# Upgrade the transformers library
#!pip install --upgrade transformers

# Import the required libraries
#from transformers import DistilBertTokenizer

# Load the DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', truncation=True, do_lower_case=True)


In [14]:
class MultiLabelDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = self.data.labels
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = self.text[index]
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [15]:
# Creating the dataset and dataloader for the neural network

train_size = 0.8
train_data=new_df.sample(frac=train_size,random_state=200)
test_data=new_df.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)


print("FULL Dataset: {}".format(new_df.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = MultiLabelDataset(train_data, tokenizer, MAX_LEN)
testing_set = MultiLabelDataset(test_data, tokenizer, MAX_LEN)

FULL Dataset: (51210, 2)
TRAIN Dataset: (40968, 2)
TEST Dataset: (10242, 2)


In [16]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [17]:
import torch
from transformers import DistilBertModel, DistilBertTokenizer
import torch.nn as nn
import torch.nn.functional as F

class DistilBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistilBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.1)
        self.additional_fc1 = torch.nn.Linear(768, 1024)  # New fully connected layer with size 256
        self.additional_fc2 = torch.nn.Linear(1024, 512)   # Another new fully connected layer with size 64
        self.classifier = torch.nn.Linear(512, 57)         # Modify output size based on your requirements

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.Tanh()(pooler)
        pooler = self.dropout(pooler)

        # Additional fully connected layers
        pooler = F.relu(self.additional_fc1(pooler))
        pooler = F.leaky_relu(self.additional_fc2(pooler))

        output = self.classifier(pooler)
        return output

# Create an instance of the modified model
#model1 = DistilBERTClass() 

#model1.to(device)

In [18]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [19]:
optimizer = torch.optim.Adam(params =  model1.parameters(), lr=LEARNING_RATE)

NameError: name 'model1' is not defined

In [21]:
def train(epoch):
    model1.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model1(ids, mask, token_type_ids)
        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%1000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
    
        loss.backward()
        optimizer.step()

In [ ]:
for epoch in range(EPOCHS):
    train(epoch)
    

In [22]:
model2 = DistilBERTClass()
model2.load_state_dict(torch.load('/kaggle/input/bert-trained-83/bert_model_trained_epoch_83.pt'))


<All keys matched successfully>

In [ ]:
# Saving the files for inference

output_model_file = './pytorch_distilbert_news.bin'
output_vocab_file = './vocab_distilbert_news.bin'

torch.save(model1, output_model_file)
tokenizer.save_vocabulary(output_vocab_file)

print('Saved')

In [23]:
#Combined Code Block
#Formatting the test data
import numpy as np
import pandas as pd
test_df = pd.read_csv('/kaggle/input/test-data1/test.csv')
test_processed = pd.DataFrame()
test_processed["Id"] = test_df['Id']
test_df['Title'] = test_df['Title'].str.lower()
test_df['Abstract'] = test_df['Abstract'].str.lower()
test_processed['Text'] = " " + test_df['Title'] + '\n' + test_df['Abstract']
articles = ['a', 'an', 'the', 'this', 'by', 'that', 'there', 'of', 'on', 'in', 'or', 'and', 'at', 'but', 'therefore', 'henceforth', 'to', 'aims', 'discusses', 'presents', 'considers', 'analyzes', 'explains', 'covers', 'deals', 'with', 'about', 'plays', 'has', 'much', 'been', 'attention', 'have', 'shown', 'several', 'efforts', 'for', 'to', 'it', ',', '.', '!', '', '$', '%', '#', '@']
def remove_words(text):
    for word in articles:
        text = text.replace(f' {word} ', ' ')
    return text.strip()
test_processed['Text'] = test_processed['Text'].apply(remove_words)
#print(test_processed)
common_stop_words = [
    'copyright', 'study', 'researcher', 'materials', 'proceedings', 'university', 'case', 'fax', 'keywords', 'discussed',
    'validates', 'has', 'there', 'dealing', 'has', 'well', 'additionally', 'volume', 'that', 'new', 'several', 'been', 'also',
    'however', 'attention', 'discussion', 'doi', 'validates', 'conclusion', 'address', 'published', 'has', 'several', 'therefore',
    'et', 'evaluating', 'determines', 'novelty', 'his', 'theirs', 'deals', 'solve', 'could', 'shown', 'considers', 'attention',
    'our', 'been', 'discussion', 'being', 'volume', 'concluding', 'aims', 'shows', 'journal', 'thus', 'study', 'it', 'much', 'solving',
    'towards', 'myself', 'have', 'reproduce', 'analyzing', 'at', 'discussing', 'propose', 'but', 'considering', 'documents', 'about',
    'issue', 'analysed', 'to', 'investigation', 'will', 'be', 'shown', 'several', 'evaluation', 'address', 'validation', 'but', 'several',
    'ours', 'hers', 'solves', 'introduction', 'which', 'use', 'novel', 'illustrates', 'shown', 'much', 'novel', 'document', 'addresses',
    'solve', 'solves', 'show', 'examining', 'experiments', 'you', 'concluding', 'has', 'would', 'been', 'evaluation', 'experiments', 'method',
    'showing', 'proposing', 'been', 'have', 'investigation', 'we', 'uses', 'has', 'been', 'there', 'experimental', 'with', 'much', 'studies', 'shown',
    'discussion', 'it', 'novelty', 'a', 'new', 'henceforth', 'investigation', 'showing', 'aim', 'results', 'shown', 'myself', 'there', 'discusses',
    'analyze', 'it', 'him', 'ourselves', 'proposing', 'dealing', 'much', 'much', 'result', 'experiment', 'there', 'been', 'reproduce', 'novelty', 'also',
    'much', 'analyzing', 'shown', 'shown', 'our', 'determine', 'that', 'conclusion', 'herself', 'documented', 'been', 'technique', 'examining', 'much',
    'investigations', 'much', 'novel', 'comparisons', 'survey', 'much', 'is', 'shown', 'uses', 'therefore', 'discusses', 'the', 'much', 'original', 'that',
    'shown', 'much', 'shown', 'experimental', 'much', 'comparisons', 'case', 'been', 'shown', 'comparisons', 'we', 'much', 'result', 'shows', 'dealing', 'shown',
    'illustrate', 'techniques', 'deals', 'conclusion', 'corresponding', 'in', 'comparisons', 'much', 'also', 'shown', 'there', 'determine', 'my', 'there', 'shown',
    'conclusively', 'comparisons', 'much', 'concluding', 'comparisons', 'we', 'validating', 'proposes', 'been', 'address', 'shown', 'deals', 'there', 'comparisons',
    'much', 'shown', 'been', 'techniques', 'ourselves', 'shown', 'much', 'concluding', 'been', 'technique', 'myself', 'there', 'experimental', 'be', 'examining', 'be',
    'been', 'case', 'a', 'much', 'discusses', 'been', 'shown', 'proposing', 'we', 'been', 'discusses', 'been', 'shown', 'been', 'technique', 'also', 'much', 'there', 'been',
    'we', 'addresses', 'we', 'been', 'there', 'been', 'my', 'validates', 'been', 'there', 'shown', 'been', 'shown', 'there', 'much', 'been', 'much', 'been', 'addresses', 'has', 'been',
    'we', 'proposing', 'been', 'there', 'been', 'there', 'been', 'techniques', 'been', 'techniques', 'techniques', 'shown', 'comparisons', 'be', 'been', 'novel', 'been', 'shown', 'been',
    'validates', 'been', 'novel', 'shown', 'addresses', 'addresses', 'proposing', 'been', 'technique', 'shown', 'been', 'techniques', 'addresses', 'addresses', 'addresses', 'there', 'techniques',
    'been', 'techniques', 'abstract', 'article', 'paper', 'title', 'author', 'authors', 'keywords', 'summary', 'introduction',
    'conclusion', 'method', 'results', 'discussion', 'proceedings', 'journal', 'volume', 'issue', 'doi', 'published',
    'published', 'conference', 'university', 'copyright', 'rights', 'reserved', 'email', 'corresponding', 'address',
    'tel', 'fax', 'et', 'al', 'figure', 'figures', 'table', 'tables', 'figure', 'figures', 'table', 'tables',
    'data', 'methodology', 'methodologies', 'study', 'studies', 'experiment', 'experiments', 'research', 'result',
    'results', 'analysis', 'discussion', 'discussion', 'discussions', 'method', 'methods', 'materials', 'material',
    'abstract', 'article', 'paper', 'title', 'author', 'authors', 'keywords', 'summary', 'introduction',
    'conclusion', 'method', 'results', 'discussion', 'proceedings', 'journal', 'volume', 'issue', 'doi', 'published',
    'published', 'conference', 'university', 'copyright', 'rights', 'reserved', 'email', 'corresponding', 'address',
    'tel', 'fax', 'et', 'al', 'figure', 'figures', 'table', 'tables', 'figure', 'figures', 'table', 'tables',
    'data', 'methodology', 'methodologies', 'study', 'studies', 'experiment', 'experiments', 'research', 'result',
    'results', 'analysis', 'discussion', 'discussion', 'discussions', 'method', 'methods', 'materials', 'material',
    'conclusion', 'conclusions', 'conclude', 'concluding', 'conclusively', 'discussion', 'discussions', 'discuss',
    'discussed', 'discussing', 'approach', 'approaches', 'used', 'using', 'use', 'based', 'case', 'study', 'study',
    'studies', 'investigate', 'investigation', 'evaluate', 'evaluation', 'assess', 'assessment', 'analyze', 'analysis',
    'propose', 'proposed', 'method', 'methods', 'technique', 'techniques', 'novel', 'novelty', 'new', 'original', 'work',
    'researcher', 'researchers', 'work', 'works', 'demonstrate', 'demonstrates', 'demonstrated', 'experimental', 'study',
    'studies', 'survey', 'surveys', 'experiment', 'experiments', 'experimental', 'validation', 'validate', 'validates',
    'validating', 'simulation', 'simulations', 'simulation', 'result', 'results', 'model', 'models', 'modeling', 'modelled',
    'model', 'analysis', 'analyses', 'analyze', 'analysed', 'analysing', 'method', 'methods', 'methodology', 'compare', 'compares',
    'comparing', 'comparison', 'comparisons', 'evaluation', 'evaluate', 'evaluates', 'evaluating', 'conclusion', 'conclusions',
    'conclude', 'concluding', 'conclusively', 'paper', 'papers', 'document', 'documents', 'documented', 'documenting', 'report',
    'reports', 'reported', 'reporting', 'investigation', 'investigations', 'study', 'studies', 'case', 'cases', 'study', 'studies',
    'show', 'shows', 'showing', 'illustrate', 'illustrates', 'illustrating', 'present', 'presents', 'presenting', 'determine',
    'determines', 'determining', 'discuss', 'discusses', 'discussing', 'propose', 'proposes', 'proposing', 'address', 'addresses',
    'addressing', 'solve', 'solves', 'solving', 'solved', 'approach', 'approaches', 'novel', 'novelty', 'new', 'original', 'use',
    'using', 'used', 'utilize', 'utilizes', 'utilizing', 'method', 'methods', 'technique', 'techniques', 'experimental', 'experiment',
    'experiments', 'survey', 'surveys', 'simulation', 'simulations', 'model', 'models', 'modeling', 'validation', 'validate',
    'validates', 'validating', 'comparison', 'comparisons', 'evaluation', 'evaluate', 'evaluates', 'evaluating', 'conclusion',
    'conclusions', 'conclude', 'concluding', 'conclusively', 'paper', 'papers', 'document', 'documents', 'report', 'reports',
    'investigation', 'investigations', 'study', 'studies', 'case', 'cases', 'show', 'shows', 'showing', 'illustrate', 'illustrates',
    'illustrating', 'present', 'presents', 'presenting', 'determine', 'determines', 'determining', 'discuss', 'discusses', 'discussing',
    'propose', 'proposes', 'proposing', 'address', 'addresses', 'addressing', 'solve', 'solves', 'solving', 'solved', 'approach',
    'approaches', 'novel', 'novelty', 'new', 'original', 'use', 'using', 'used', 'utilize', 'utilizes', 'utilizing', 'method', 'methods',
    'technique', 'techniques', 'experimental', 'experiment', 'experiments', 'survey', 'surveys', 'simulation', 'simulations', 'model',
    'models', 'modeling', 'validation', 'validate', 'validates', 'validating', 'comparison', 'comparisons', 'evaluation', 'evaluate',
    'evaluates', 'evaluating', 'conclusion', 'conclusions', 'conclude', 'concluding', 'conclusively', 'paper', 'papers', 'document',
    'documents', 'report', 'reports', 'investigation', 'investigations', 'study', 'studies', 'case', 'cases', 'show', 'shows', 'showing',
    'illustrate', 'illustrates', 'illustrating', 'present', 'presents', 'presenting', 'determine', 'determines', 'determining', 'discuss',
    'discusses', 'discussing', 'propose', 'proposes', 'proposing', 'address', 'addresses', 'addressing', 'solve', 'solves', 'solving', 'solved',
    'approach', 'approaches', 'novel', 'novelty', 'new', 'original', 'use', 'using', 'used', 'utilize', 'utilizes', 'utilizing', 'method', 'methods',
    'technique', 'techniques', 'experimental', 'experiment', 'experiments', 'survey', 'surveys', 'simulation', 'simulations', 'model', 'models',
    'modeling', 'validation', 'validate', 'validates', 'validating', 'comparison', 'comparisons', 'evaluation', 'evaluate', 'evaluates', 'evaluating',
    'conclusion', 'conclusions', 'conclude', 'concluding', 'conclusively', 'paper', 'papers', 'document', 'documents', 'report', 'reports', 'investigation',
    'investigations', 'study', 'studies', 'case', 'cases', 'show', 'shows', 'showing', 'illustrate', 'illustrates', 'illustrating', 'present', 'presents',
    'presenting', 'determine', 'determines', 'determining', 'discuss', 'discusses', 'discussing', 'propose', 'proposes', 'proposing', 'address', 'addresses',
    'addressing', 'solve', 'solves', 'solving', 'solved', 'approach', 'approaches', 'novel', 'novelty', 'new', 'original', 'use', 'using', 'used', 'utilize',
    'utilizes', 'utilizing', 'method', 'methods', 'technique', 'techniques', 'experimental', 'experiment', 'experiments', 'survey', 'surveys', 'simulation',
    'simulations', 'model', 'models', 'modeling', 'validation', 'validate', 'validates', 'validating', 'comparison', 'comparisons', 'evaluation', 'evaluate',
    'evaluates', 'evaluating', 'conclusion', 'conclusions', 'conclude', 'concluding', 'conclusively', 'paper', 'papers', 'document', 'documents', 'report',
    'reports', 'investigation', 'investigations', 'study', 'studies', 'case', 'cases', 'show', 'shows', 'showing', 'illustrate', 'illustrates', 'illustrating',
    'present', 'presents', 'presenting', 'determine', 'determines', 'determining', 'discuss', 'discusses', 'discussing', 'propose', 'proposes', 'proposing',
    'address', 'addresses', 'addressing', 'solve', 'solves', 'solving', 'solved', 'approach', 'approaches', 'novel', 'novelty', 'new', 'original', 'use', 'using',
    'used', 'utilize', 'utilizes', 'utilizing', 'method', 'methods', 'technique', 'techniques', 'experimental', 'experiment', 'experiments', 'survey', 'surveys',
    'simulation', 'simulations', 'model', 'models', 'modeling', 'validation', 'validate', 'validates', 'validating', 'comparison', 'comparisons', 'evaluation',
    'evaluate', 'evaluates', 'evaluating', 'conclusion', 'conclusions', 'conclude', 'concluding', 'conclusively', 'paper', 'papers', 'document', 'documents',
    'report', 'reports', 'investigation', 'investigations', 'study', 'studies', 'case', 'cases', 'show', 'shows', 'showing', 'illustrate', 'illustrates',
    'illustrating', 'present', 'presents', 'presenting', 'determine', 'determines', 'determining', 'discuss', 'discusses', 'discussing', 'propose', 'proposes',
    'proposing', 'address', 'addresses', 'addressing', 'solve', 'solves', 'solving', 'solved', 'approach', 'approaches', 'novel', 'novelty', 'new', 'original',
    'use', 'using', 'used', 'utilize', 'utilizes', 'utilizing', 'method', 'methods', 'technique', 'techniques', 'experimental', 'experiment', 'experiments',
    'survey', 'surveys', 'simulation', 'simulations', 'model', 'models', 'modeling', 'validation', 'validate', 'validates', 'validating', 'comparison',
    'comparisons', 'evaluation', 'evaluate', 'evaluates', 'evaluating', 'conclusion', 'conclusions', 'conclude', 'concluding', 'conclusively', 'paper',
    'papers', 'document', 'documents', 'report', 'reports', 'investigation', 'investigations', 'study', 'studies', 'case', 'cases', 'show', 'shows',
    'showing', 'illustrate', 'illustrates', 'illustrating', 'present', 'presents', 'presenting', 'determine', 'determines', 'determining', 'discuss',
    'discusses', 'discussing', 'propose', 'proposes', 'proposing', 'address', 'addresses', 'addressing', 'solve', 'solves', 'solving', 'solved', 'approach',
    'approaches', 'novel', 'novelty', 'new', 'original', 'use', 'using', 'used', 'utilize', 'utilizes', 'utilizing', 'method', 'methods', 'technique', 'techniques',
    'experimental', 'experiment', 'experiments', 'survey', 'surveys', 'simulation', 'simulations', 'model', 'models', 'modeling', 'validation', 'validate',
    'validates', 'validating', 'comparison', 'comparisons', 'evaluation', 'evaluate', 'evaluates', 'evaluating', 'conclusion', 'conclusions', 'conclude',
    'concluding', 'conclusively', 'paper', 'papers', 'document', 'documents', 'report', 'reports', 'investigation', 'investigations', 'study', 'studies', 'case',
    'cases', 'show', 'shows', 'showing', 'illustrate', 'illustrates', 'illustrating', 'present', 'presents', 'presenting', 'determine', 'determines', 'determining',
    'discuss', 'discusses', 'discussing', 'propose', 'proposes', 'proposing', 'address', 'addresses', 'addressing', 'solve', 'solves', 'solving', 'solved', 'approach',
    'approaches', 'novel', 'novelty', 'new', 'original', 'use', 'using', 'used', 'utilize', 'utilizes', 'utilizing', 'method', 'methods', 'technique', 'techniques',
    'experimental', 'experiment', 'experiments', 'survey', 'surveys', 'simulation', 'simulations', 'model', 'models', 'modeling', 'validation', 'validate',
    'validates', 'validating', 'comparison', 'comparisons', 'evaluation', 'evaluate', 'evaluates', 'evaluating', 'conclusion', 'conclusions', 'conclude',
    'concluding', 'conclusively', 'paper', 'papers', 'document', 'documents', 'report', 'reports', 'investigation', 'investigations', 'study', 'studies', 'case',
    'cases', 'show', 'shows', 'showing', 'illustrate', 'illustrates', 'illustrating', 'present', 'presents', 'presenting', 'determine', 'determines', 'determining',
    'discuss', 'discusses', 'discussing', 'propose']
common_stop_words = list(set(common_stop_words))
def remove_words(text):
    for word in common_stop_words:
        text = text.replace(f' {word} ', ' ')
    return text.strip()
test_processed['Text'] = test_processed['Text'].apply(remove_words)
print(test_processed)

          Id                                               Text
0      30332  pricing fx options under intermediate currency...
1      50337  multicore processor real-time system automobil...
2      66515  perceptual quality improvement videoconferenci...
3      57464  hundred-kilobyte lookup efficient single-image...
4      43169  efficient sequence labeling actor-critic train...
...      ...                                                ...
10969  41708  new lipid-structured opposing effects ldl hdl ...
10970  38843  evaluating efficacy hybrid deep learning disti...
10971  57571  weakly supervised video individual countingwea...
10972  31964  structformer: learning spatial structure langu...
10973  22229  numerical guided lazy abstraction refinement n...

[10974 rows x 2 columns]


In [34]:
def validation(testing_loader):
    model2.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to('device', dtype = torch.long)
            mask = data['mask'].to('device', dtype = torch.long)
            token_type_ids = data['token_type_ids'].to('device', dtype = torch.long)
            targets = data['targets'].to('device', dtype = torch.float)
            outputs = model2(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [35]:
outputs, targets = validation(testing_loader)

2561it [1:03:12,  1.48s/it]


In [41]:
outputs.head()

AttributeError: 'list' object has no attribute 'head'

In [37]:
output1 = outputs

In [40]:
#SIGMOIDING the outputs
op = 1 / (1 + np.exp( - np.array(output1)))

In [43]:
op = pd.DataFrame(op)

In [44]:
op.describe()

,0,1,2,3,4,5,6,7,8,9,...,47,48,49,50,51,52,53,54,55,56
count,10242.000000,10242.000000,10242.000000,10242.000000,10242.000000,10242.000000,10242.000000,10242.000000,10242.000000,10242.000000,...,10242.000000,10242.000000,10242.000000,10242.000000,10242.000000,10242.000000,10242.000000,10242.000000,10242.000000,10242.000000
mean,0.541570,0.502014,0.500731,0.532622,0.514307,0.536756,0.503577,0.505952,0.501319,0.502055,...,0.503027,0.501593,0.502194,0.502090,0.501386,0.502815,0.502078,0.509999,0.519345,0.516117
std,0.083913,0.020789,0.011803,0.079214,0.054465,0.083478,0.027180,0.034794,0.015892,0.020702,...,0.024902,0.018099,0.021696,0.020703,0.016628,0.022919,0.020304,0.045017,0.061649,0.057740
min,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
25%,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
50%,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
75%,0.503861,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
max,0.731059,0.731059,0.731041,0.731059,0.731059,0.731059,0.731059,0.731059,0.731058,0.731059,...,0.731058,0.731058,0.731059,0.731059,0.731059,0.731058,0.731059,0.731059,0.731059,0.731059


In [121]:
trial = op

In [47]:
final_outputs = np.array(trial) >=0.6

In [48]:
np.shape(outputs)

(10242, 57)

In [49]:
final2 = outputs
len(final2)

10242

In [50]:
final1 = outputs

In [51]:
len(final1)

10242

In [119]:
final1000 = pd.DataFrame(final1)

In [120]:
final1000.apply(pd.Series.describe, axis=1)

,count,mean,std,min,25%,50%,75%,max
0,57.0,0.035078,0.185583,0.000000e+00,4.638385e-31,2.091244e-22,1.880283e-18,0.999930
1,57.0,0.044281,0.191730,0.000000e+00,8.590584e-27,2.945238e-21,7.111949e-12,0.999963
2,57.0,0.000531,0.002661,0.000000e+00,8.188473e-27,5.403606e-20,6.594332e-13,0.019497
3,57.0,0.017544,0.132453,0.000000e+00,3.963515e-25,6.269326e-18,9.383820e-15,1.000000
4,57.0,0.056159,0.225396,4.309365e-34,8.403732e-18,1.777381e-13,2.328467e-07,1.000000
...,...,...,...,...,...,...,...,...
10237,57.0,0.020719,0.134181,0.000000e+00,4.475856e-33,1.332008e-27,1.243388e-20,1.000000
10238,57.0,0.039556,0.172488,0.000000e+00,5.083258e-25,1.758317e-16,1.213713e-10,1.000000
10239,57.0,0.075662,0.246204,5.859001e-29,2.977592e-21,5.951741e-15,4.663563e-11,1.000000
10240,57.0,0.030175,0.144440,1.443384e-35,1.148846e-19,1.669244e-15,5.438104e-10,0.999947


In [52]:
print(final1[:5])

[[0.9994876384735107, 8.271194381977581e-35, 1.3914468304897508e-17, 0.9999302625656128, 1.0700975677018354e-21, 1.4550254040903224e-14, 4.7334576755289555e-17, 2.013239458156143e-30, 2.674939815347778e-26, 2.0912435015789972e-22, 5.395524064561114e-15, 3.0496841824387354e-26, 7.678221663809381e-06, 2.082145558941241e-14, 2.098049823915428e-31, 0.0, 5.67800067203385e-22, 1.0048428522580791e-19, 2.611197955812152e-21, 1.6520492552939696e-18, 2.9776710769674293e-21, 1.6684669706557996e-16, 3.474723564840834e-28, 1.880283275548376e-18, 1.169006327302359e-31, 3.8876995360366924e-32, 4.638384569502885e-31, 7.934896128467121e-22, 7.615450556337788e-28, 5.03990447850171e-35, 2.4525431555356146e-37, 5.100813912151017e-34, 1.1494864493715014e-25, 5.914401560374885e-21, 2.50485715848099e-38, 5.154702577531849e-27, 9.363129336761265e-35, 7.475369197390749e-27, 4.3334533145052576e-17, 2.359188715137041e-31, 7.118189348370458e-24, 2.1735699753998392e-15, 1.0063081897885197e-19, 1.1794782014309056e-

In [125]:
trial.apply(pd.Series.describe, axis = 1)

,count,mean,std,min,25%,50%,75%,max
0,57.0,0.508105,0.042882,0.5,0.5,0.5,0.5,0.731045
1,57.0,0.510384,0.044535,0.5,0.5,0.5,0.5,0.731051
2,57.0,0.500133,0.000665,0.5,0.5,0.5,0.5,0.504874
3,57.0,0.504054,0.030604,0.5,0.5,0.5,0.5,0.731059
4,57.0,0.513042,0.052107,0.5,0.5,0.5,0.5,0.731058
...,...,...,...,...,...,...,...,...
10237,57.0,0.504845,0.031077,0.5,0.5,0.5,0.5,0.731059
10238,57.0,0.509393,0.040692,0.5,0.5,0.5,0.5,0.731059
10239,57.0,0.517793,0.057683,0.5,0.5,0.5,0.5,0.731059
10240,57.0,0.507186,0.033798,0.5,0.5,0.5,0.5,0.731048


In [149]:
final1_1 = trial >= 0.608
final1_2 = trial >= 0.02
final1_3 = trial >= 0.03
final1_4 = trial >= 0.04
final1_5 = trial >= 0.05
final1_6 = trial >= 0.06
final1_7 = trial >= 0.07
final1_8 = trial >= 0.08
final1_9 = trial >= 0.09
final1_10 = trial >= 0.1

In [151]:
final1_1 = final1_1 * 1
final1_2 = final1_2 * 1
final1_3 = final1_3 * 1
final1_4 = final1_4 * 1
final1_5 = final1_5 * 1
final1_6 = final1_6 * 1
final1_7 = final1_7 * 1
final1_8 = final1_8 * 1
final1_9 = final1_9 * 1
final1_10 = final1_10 * 1
final1_10

,0,1,2,3,4,5,6,7,8,9,...,47,48,49,50,51,52,53,54,55,56
0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10237,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
10238,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
10239,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
10240,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [80]:
f = final1_1.sum(axis = 1)

In [81]:
f.describe()

count    10242.000000
mean         2.099199
std          1.262810
min          0.000000
25%          1.000000
50%          2.000000
75%          3.000000
max          8.000000
dtype: float64

In [82]:
from sklearn.metrics import f1_score

In [152]:
print(f1_score(targets, final1_1, average= 'macro'))
print(f1_score(targets, final1_2, average= 'macro'))
print(f1_score(targets, final1_3, average= 'macro'))
print(f1_score(targets, final1_4, average= 'macro'))
print(f1_score(targets, final1_5, average= 'macro'))
print(f1_score(targets, final1_6, average= 'macro'))
print(f1_score(targets, final1_7, average= 'macro'))
print(f1_score(targets, final1_8, average= 'macro'))
print(f1_score(targets, final1_9, average= 'macro'))
print(f1_score(targets, final1_10, average= 'macro'))

0.6508610092534152
0.06573937896188514
0.06573937896188514
0.06573937896188514
0.06573937896188514
0.06573937896188514
0.06573937896188514
0.06573937896188514
0.06573937896188514
0.06573937896188514


In [153]:
print(final1_1.describe())

                 0             1             2             3             4   \
count  10242.000000  10242.000000  10242.000000  10242.000000  10242.000000   
mean       0.180238      0.008592      0.003027      0.141476      0.061707   
std        0.384405      0.092299      0.054935      0.348529      0.240634   
min        0.000000      0.000000      0.000000      0.000000      0.000000   
25%        0.000000      0.000000      0.000000      0.000000      0.000000   
50%        0.000000      0.000000      0.000000      0.000000      0.000000   
75%        0.000000      0.000000      0.000000      0.000000      0.000000   
max        1.000000      1.000000      1.000000      1.000000      1.000000   

                 5             6             7             8             9   \
count  10242.000000  10242.000000  10242.000000  10242.000000  10242.000000   
mean       0.159344      0.015231      0.025679      0.005370      0.008787   
std        0.366014      0.122478      0.158182    

In [157]:
final1_1.sum()

0     1846
1       88
2       31
3     1449
4      632
5     1632
6      156
7      263
8       55
9       90
10     290
11     696
12    2630
13     288
14     293
15      16
16     260
17     417
18     598
19     282
20     124
21      88
22      49
23     628
24     244
25     336
26     190
27     499
28     106
29     575
30     150
31     178
32     695
33     149
34     318
35     563
36     353
37     709
38      95
39      29
40      54
41      67
42      44
43      36
44      99
45     104
46      43
47     136
48      69
49      98
50      94
51      61
52     123
53      91
54     445
55     855
56     713
dtype: int64

In [ ]:
THRESHOLD = 0.1
final_outputs = np.zeros(np.shape(outputs))
for i in tqdm(range(np.shape(outputs)[0])):
    final_outputs[i][np.argmax(outputs, axis = 1)] = THRESHOLD
    #print(final_outputs[i])
final_outputs = np.array(outputs) >= THRESHOLD

In [ ]:
final_outputs

In [ ]:
final_outputs
final_outputs1 = pd.DataFrame(final_outputs * 1)

In [ ]:
print(f1_score(targets, final_outputs1, average= 'macro'))

In [84]:
#testing on the test set
test_processed

,Id,Text
0,30332,pricing fx options under intermediate currency...
1,50337,multicore processor real-time system automobil...
2,66515,perceptual quality improvement videoconferenci...
3,57464,hundred-kilobyte lookup efficient single-image...
4,43169,efficient sequence labeling actor-critic train...
...,...,...
10969,41708,new lipid-structured opposing effects ldl hdl ...
10970,38843,evaluating efficacy hybrid deep learning disti...
10971,57571,weakly supervised video individual countingwea...
10972,31964,structformer: learning spatial structure langu...


In [85]:
from torch.utils.data import DataLoader

# Assuming you have a dataset for your unlabelled test data (test_dataset)
test_loader = DataLoader(test_processed, batch_size=VALID_BATCH_SIZE, shuffle=False)

In [94]:
from torch import cuda
#device = 'cpu' #if cuda.is_available() else 'cpu'

def predict(model, dataloader):
    model2.eval()
    all_predictions = []
    #all_indices = []

    with torch.no_grad():
        for batch_index, data in tqdm(enumerate(dataloader)):
            ids = data['ids'].to(device, dtype=torch.long).int()
            mask = data['mask'].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)

            outputs = model2(ids, mask, token_type_ids)

            predictions = outputs  # Assuming it's a binary classification problem, adjust as needed
            all_predictions.append(predictions.cpu().numpy())
            #all_indices.append(data['index'].cpu().numpy())  # Assuming 'index' is a field in your dataset

    all_predictions = np.vstack(all_predictions)
    #all_indices = np.concatenate(all_indices)
    #sorted_indices = np.argsort(all_indices)
    return all_predictions


In [96]:
test_final = pd.DataFrame()
test_processed
test_final['text'] = test_processed['Text']
test_final

,text
0,pricing fx options under intermediate currency...
1,multicore processor real-time system automobil...
2,perceptual quality improvement videoconferenci...
3,hundred-kilobyte lookup efficient single-image...
4,efficient sequence labeling actor-critic train...
...,...
10969,new lipid-structured opposing effects ldl hdl ...
10970,evaluating efficacy hybrid deep learning disti...
10971,weakly supervised video individual countingwea...
10972,structformer: learning spatial structure langu...


In [97]:
class PredictLabel(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = self.text[index]
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
        }

In [98]:
#test_data=new_df.drop(train_data.index).reset_index(drop=True)
testing_data = test_final#.reset_index(drop=True)

testing_set = PredictLabel(testing_data, tokenizer, MAX_LEN)
#testing_set = MultiLabelDataset(test_data, tokenizer, MAX_LEN)

In [99]:
test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }
testing_loader = DataLoader(testing_set, **test_params)

In [100]:
unlabelled_predictions = predict(model2, testing_loader)

2744it [1:02:25,  1.36s/it]


In [1]:
unlabelled_predictions = pd.read_csv('/kaggle/input/rangari-tfidf/preds.csv')

In [5]:
unlabelled_predictions = unlabelled_predictions * 1000000

In [6]:
unlabelled_predictions.apply(pd.Series.describe, axis = 1)

,count,mean,std,min,25%,50%,75%,max
0,57.0,0.362018,0.099916,0.30900,0.32500,0.33700,0.35700,1.0100
1,57.0,0.007706,0.002927,0.00609,0.00664,0.00698,0.00747,0.0270
2,57.0,0.004273,0.001698,0.00333,0.00366,0.00384,0.00415,0.0155
3,57.0,0.003940,0.001580,0.00307,0.00337,0.00354,0.00382,0.0144
4,57.0,0.362018,0.099916,0.30900,0.32500,0.33700,0.35700,1.0100
...,...,...,...,...,...,...,...,...
10969,57.0,0.007706,0.002927,0.00609,0.00664,0.00698,0.00747,0.0270
10970,57.0,0.004273,0.001698,0.00333,0.00366,0.00384,0.00415,0.0155
10971,57.0,0.003940,0.001580,0.00307,0.00337,0.00354,0.00382,0.0144
10972,57.0,0.362018,0.099916,0.30900,0.32500,0.33700,0.35700,1.0100


In [7]:
unlabelled_predictions.describe()

,0,1,2,3,4,5,6,7,8,9,...,47,48,49,50,51,52,53,54,55,56
count,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,...,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000
mean,0.138221,0.082404,0.087093,0.126179,0.093671,0.148948,0.084370,0.091736,0.085432,0.087375,...,0.086828,0.090001,0.086488,0.081809,0.085650,0.080772,0.089221,0.089256,0.114955,0.094571
std,0.224511,0.134895,0.142582,0.205478,0.153220,0.241993,0.138379,0.150295,0.140076,0.142996,...,0.142158,0.147255,0.141776,0.134083,0.140527,0.132373,0.145973,0.145953,0.187128,0.154433
min,0.006430,0.003350,0.003530,0.005630,0.003860,0.006910,0.003300,0.003670,0.003370,0.003570,...,0.003520,0.003690,0.003420,0.003250,0.003330,0.003210,0.003660,0.003700,0.005160,0.004020
25%,0.006960,0.003630,0.003840,0.006100,0.004190,0.007480,0.003590,0.003980,0.003660,0.003870,...,0.003820,0.004010,0.003720,0.003530,0.003620,0.003490,0.003970,0.004010,0.005590,0.004360
50%,0.012400,0.006580,0.006940,0.010900,0.007570,0.013300,0.006530,0.007230,0.006640,0.007000,...,0.006910,0.007240,0.006750,0.006400,0.006590,0.006330,0.007190,0.007250,0.009990,0.007840
75%,0.398350,0.238645,0.252235,0.364225,0.271143,0.429325,0.244633,0.265807,0.247660,0.253000,...,0.251477,0.260560,0.250688,0.237100,0.248397,0.234082,0.258298,0.258313,0.331747,0.273460
max,0.527000,0.316000,0.334000,0.482000,0.359000,0.568000,0.324000,0.352000,0.328000,0.335000,...,0.333000,0.345000,0.332000,0.314000,0.329000,0.310000,0.342000,0.342000,0.439000,0.362000


In [8]:
final11 = unlabelled_predictions


In [9]:
save = unlabelled_predictions

In [10]:
trayy = save

In [23]:
final11_1 = final11 >= 0.365
final11_2 = final11 >= 0.37
final11_3 = final11 >= 0.375

In [24]:
final11_1 = final11_1 *1 
final11_2 = final11_2 *1
final11_3 = final11_3 *1

In [25]:
print(final11_1.sum().sum())
print(final11_2.sum().sum())
print(final11_3.sum().sum())

32928
27440
24696


In [26]:
columns = pd.read_csv('/kaggle/input/data-columns/Processed_Data.csv')

In [27]:
final11_1.columns = columns.columns[4:]
final11_2.columns = columns.columns[4:]
final11_3.columns = columns.columns[4:]

In [29]:
final11_3

,cs.AI,cs.AR,cs.CE,cs.CL,cs.CR,cs.CV,cs.DB,cs.DC,cs.DM,cs.GT,...,q-fin.MF,q-fin.PM,q-fin.PR,q-fin.RM,q-fin.TR,stat.AP,stat.CO,stat.ME,stat.ML,stat.TH
0,1,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10969,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10970,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10971,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10972,1,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [30]:
final11_1['Id'] = test_processed['Id']
final11_1

,cs.AI,cs.AR,cs.CE,cs.CL,cs.CR,cs.CV,cs.DB,cs.DC,cs.DM,cs.GT,...,q-fin.PM,q-fin.PR,q-fin.RM,q-fin.TR,stat.AP,stat.CO,stat.ME,stat.ML,stat.TH,Id
0,1,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,30332
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,50337
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,66515
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,57464
4,1,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,43169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10969,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,41708
10970,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,38843
10971,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,57571
10972,1,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,31964


In [31]:
final11_2['Id'] = test_processed['Id']
final11_2

,cs.AI,cs.AR,cs.CE,cs.CL,cs.CR,cs.CV,cs.DB,cs.DC,cs.DM,cs.GT,...,q-fin.PM,q-fin.PR,q-fin.RM,q-fin.TR,stat.AP,stat.CO,stat.ME,stat.ML,stat.TH,Id
0,1,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,30332
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,50337
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,66515
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,57464
4,1,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,43169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10969,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,41708
10970,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,38843
10971,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,57571
10972,1,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,31964


In [32]:
final11_3['Id'] = test_processed['Id']
final11_3

,cs.AI,cs.AR,cs.CE,cs.CL,cs.CR,cs.CV,cs.DB,cs.DC,cs.DM,cs.GT,...,q-fin.PM,q-fin.PR,q-fin.RM,q-fin.TR,stat.AP,stat.CO,stat.ME,stat.ML,stat.TH,Id
0,1,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,30332
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,50337
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,66515
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,57464
4,1,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,43169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10969,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,41708
10970,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,38843
10971,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,57571
10972,1,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,31964


In [33]:
sample_cols = pd.read_csv('/kaggle/input/sample-columns/sample_submission.csv')

In [34]:
sample_cols.columns

Index(['Id', 'math.AT', 'stat.AP', 'cs.AR', 'math.QA', 'q-bio.MN', 'eess.AS',
       'eess.IV', 'stat.ME', 'econ.GN', 'eess.SP', 'q-fin.RM', 'cs.LG',
       'cs.CR', 'q-bio.BM', 'q-fin.GN', 'q-fin.MF', 'q-fin.PR', 'math.CV',
       'cs.LO', 'econ.TH', 'math.CO', 'cs.AI', 'math.AC', 'q-bio.CB',
       'q-fin.CP', 'cs.CL', 'cs.DC', 'math.LO', 'math.NT', 'cs.SD', 'q-fin.TR',
       'cs.CV', 'stat.ML', 'q-fin.EC', 'econ.EM', 'cs.CE', 'stat.CO',
       'math.PR', 'q-bio.NC', 'math.AP', 'cs.OS', 'cs.NI', 'cs.IT', 'cs.PL',
       'cs.GT', 'cs.DM', 'math.IT', 'cs.SE', 'cs.RO', 'stat.TH', 'cs.DB',
       'math.ST', 'q-bio.GN', 'q-fin.PM', 'q-bio.TO', 'math.GR', 'cs.IR'],
      dtype='object')

In [35]:
final11_1= final11_1[sample_cols.columns]
final11_1

,Id,math.AT,stat.AP,cs.AR,math.QA,q-bio.MN,eess.AS,eess.IV,stat.ME,econ.GN,...,cs.SE,cs.RO,stat.TH,cs.DB,math.ST,q-bio.GN,q-fin.PM,q-bio.TO,math.GR,cs.IR
0,30332,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,50337,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,66515,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,57464,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,43169,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10969,41708,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10970,38843,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10971,57571,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10972,31964,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [36]:
final11_2= final11_2[sample_cols.columns]
final11_2

,Id,math.AT,stat.AP,cs.AR,math.QA,q-bio.MN,eess.AS,eess.IV,stat.ME,econ.GN,...,cs.SE,cs.RO,stat.TH,cs.DB,math.ST,q-bio.GN,q-fin.PM,q-bio.TO,math.GR,cs.IR
0,30332,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,50337,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,66515,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,57464,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,43169,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10969,41708,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10970,38843,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10971,57571,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10972,31964,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [37]:
final11_3 = final11_3[sample_cols.columns]


In [38]:
final11_3

,Id,math.AT,stat.AP,cs.AR,math.QA,q-bio.MN,eess.AS,eess.IV,stat.ME,econ.GN,...,cs.SE,cs.RO,stat.TH,cs.DB,math.ST,q-bio.GN,q-fin.PM,q-bio.TO,math.GR,cs.IR
0,30332,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,50337,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,66515,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,57464,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,43169,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10969,41708,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10970,38843,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10971,57571,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10972,31964,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [182]:
final11_1.to_csv('submission11_111.csv', index = False)

In [39]:
final11_2.to_csv('rangari1.csv', index = False)

In [40]:
final11_3.to_csv('rangari2.csv', index = False)